# Карта каналов
Пример расчета половозрастной структуры каналов с построением графика

## Описание задачи и условий расчета

- Период: 01.08.2023 - 31.08.2023
- Временной интервал: 05:00-29:00
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы 1 и 2 мультиплексов										
- Статистики: TgSat%, WAgeAverage, Share

Для работы с данным ноутбуком нам потребуется библиотека **plotly**. Выполните следующую ячейку для установки:

In [ ]:
!pip install plotly

Библиотека устанавливается один раз. При последующих сессиях ячейку с установкой можно пропускать (или вообще удалить)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

import plotly.express as px

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике демографических атрибутов найдем необходимые нам переменные
cats.get_tv_demo_attribute(names=['пол'])

# Для формирования условий запроса нам понадобятся значение столбца entityName: sex

In [ ]:
# В справочнике телесетей получим необходимые идентификаторы

# Загружаем справочник
nets = cats.get_tv_net()

# Фильтруем по списку каналов и выводим список id
nets[nets['name'].isin(['ДОМАШНИЙ','ЗВЕЗДА','КАРУСЕЛЬ','МАТЧ ТВ','МИР','МУЗ ТВ','НТВ','ПЕРВЫЙ КАНАЛ','ПЯТНИЦА',
                       'ПЯТЫЙ КАНАЛ','РЕН ТВ','РОССИЯ 1','РОССИЯ 24','РОССИЯ К','СТС','ТВ-3','ТВ ЦЕНТР',
                       'ТНТ','СПАС'])]['id'].values

## Расчет заданий
Нам потребуется два расчета:

### 1. Средний возраст и доля
Посчитаем средний возраст и долю для каждого канала

#### Формирование задания

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2023-08-01', '2023-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы: 1 и 2 мультиплексы
company_filter = 'tvNetId IN (1,2,4,10,11,12,13,40,60,83,86,204,206,257,258,259,260,326,393,430)'

# Указываем список статистик для расчета
statistics = ['WAgeAverage', 'Share']

# Указываем срезы: телесети
slices = ['tvNetName']

# Задаем условия сортировки: телесеть (от а до я)
sortings = {"tvNetName":"ASC"}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "totalType": "TotalChannels" #база расчета Share: Total Channels. Возможны опции: TotalTVSet, TotalChannelsThem
}

#### Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter,location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df_1 = mtask.result2table(mtask.get_result(task_timeband), project_name='Total. Ind')

# Приводим порядок столбцов в соответствие с условиями расчета
df_1 = df_1[['prj_name']+slices+statistics]

# Округляем значения статистик
df_1 = df_1.round({'WAgeAverage':0, 'Share':3})
df_1

### 2. Процент женской аудитории
Посчитаем процент женщин в аудитории каждого канала

#### Формирование задания

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2023-08-01', '2023-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА: женщины
targetdemo_filter = 'sex = 2'

# Задаем место просмотра
location_filter=None

# Задаем каналы: 1 и 2 мультиплексы
company_filter = 'tvNetId IN (1,2,4,10,11,12,13,40,60,83,86,204,206,257,258,259,260,326,393,430)'

# Указываем список статистик для расчета
statistics = ['TgSatPer']

# Указываем срезы: телесети
slices = ['tvNetName']

# Задаем условия сортировки: телесеть (от а до я)
sortings = {"tvNetName":"ASC"}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
}

#### Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter,location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df_2 = mtask.result2table(mtask.get_result(task_timeband), project_name='Пол(женский)')

# Приводим порядок столбцов в соответствие с условиями расчета
df_2 = df_2[['prj_name']+slices+statistics]

# Округляем значения статистик
df_2 = df_2.round({'TgSatPer':1})
df_2

### Объединим в одну таблицу

In [ ]:
df = pd.concat([df_2, df_1])

In [ ]:
# Формируем сводную таблицу: строки - каналы, столбцы - ЦА, значения - статистики
df = pd.pivot_table(df, values = ['TgSatPer','WAgeAverage','Share'],
                        index=['tvNetName'], 
                        columns=['prj_name'])

# Меняем местами уровни в столбцах
df = df.swaplevel(0,1, axis=1)

# подготовим датафрейм (он не должен быть мультииндексным, должен быть с обычными плоскими колонками)
df = df.reset_index()
df.columns = [' '.join(col).strip() for col in df.columns.values]
df

### Построим график карты каналов

In [ ]:
# указываем названия полей из датасета и сам датафрейм
dataframe = df
X = "Пол(женский) TgSatPer"
Y = "Total. Ind WAgeAverage"
SIZE = "Total. Ind Share"
NAMES = "tvNetName"

fig = px.scatter(dataframe,
                 x=X,
                 y=Y,
                 size=SIZE,
                 hover_name=NAMES,
                 text=NAMES,
                 size_max=80,
                 title="Половозрастная структура телеканалов мультиплекса по доле аудитории",
                 labels={
                     X: "% женщин в аудитории",
                     Y: "средний возраст зрителей",
                     SIZE: "Share",
                     NAMES: "Телеканал"
                 },
                 range_x=[20, 90], #минимальное и максимальное значение оси x (% женщин)
                 range_y=[20, 70], #минимальное и максимальное значение оси y (средний возраст)
                 height=600 #размер графика
                 )

# меняем цвет
fig.update_traces(marker=dict(color='#22a98b'))

# добавляем линии средних значений
fig.add_hline(y=41.6, line_color='gray', line_width=0.5)
fig.add_vline(x=53.7, line_color='gray', line_width=0.5)

fig.update_traces(textposition='middle center')

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 10, #шаг значений по оси x (% женщин)
        showgrid = False
    ),
    yaxis = dict(
        tickmode = 'linear',
        dtick = 5, #шаг значений по оси y (средний возраст)
        showgrid = False
    )
)

fig.update_layout(
    plot_bgcolor='white'
)

fig.show()

#### Как читать график:
- Горизонтальная ось показывает % женщин в аудитории, а вертикальная ось – средний возраст зрителей всего ТВ

- Точка пересечения линий на графике соответствует среднему половозрастному распределению генеральной совокупности: 53,7% женщин в населении, средний возраст – 41,6 год

- Чем выше на графике расположен шар, тем старше аудитория соответствующего ему жанра

- Чем правее расположен шар, тем больше женщин в его аудитории

- Размер шара соответствует доле аудитории соответствующего канала

График интерактивный: 
- при наведении на шар канала появляется тултип с данными
- возможно изменение масштаба (zoom in, zoom out)
- для сохранения графика в png нажмите на иконку фотоаппарата 

## Экспорт в Excel
Сохраним таблицу с данными

По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('04_channels_map')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)